In [5]:
import pandas as pd

#Utilisation d'un Modèle Pré-Entrraîné
#Le fine-tuning consiste à prendre un modèle de langage pré-entraîné sur un vaste corpus de données générales et à l'ajuster 
# (fine-tuner) sur un ensemble de données spécifique à votre tâche. Pour la classification de texte, vous pouvez utiliser 
# des modèles comme BERT, DistilBERT, ou RoBERTa.

#2. Avantages du Fine-Tuning
#Connaissances Générales : Le modèle pré-entraîné a déjà appris une grande quantité de connaissances générales sur le langage, 
# ce qui lui permet de mieux comprendre les nuances du texte.
#Adaptation Spécifique : En fine-tunant le modèle sur vos données spécifiques, vous pouvez l'adapter pour qu'il fasse des 
# prédictions précises pour votre tâche particulière.

data = {
    "note": [
        "Le serveur est en panne, tout est bloqué.",
        "L'application est lente à charger.",
        "Une fonctionnalité mineure ne fonctionne pas.",
        "La base de données est corrompue.",
        "La page de connexion prend du temps à s'afficher.",
        "Le bouton de téléchargement ne fonctionne pas.",
        "Le réseau est complètement hors service.",
        "Il y a un léger décalage dans l'affichage.",
        "Le système redémarre de manière aléatoire.",
        "Certains utilisateurs ne peuvent pas se connecter.",
        "Les emails ne sont pas envoyés correctement.",
        "Le site web est inaccessible pour certains utilisateurs.",
        "Les notifications push ne fonctionnent pas.",
        "Les rapports ne sont pas générés comme prévu.",
        "Le paiement en ligne ne fonctionne pas.",
        "Le temps de réponse du serveur est très lent.",
        "Des erreurs 500 apparaissent fréquemment.",
        "L'interface utilisateur est boguée.",
        "Des fichiers sont manquants dans la base de données.",
        "Les utilisateurs sont déconnectés automatiquement.",
        "Le système de sauvegarde ne fonctionne pas.",
        "Le processus de login est très lent.",
        "Des erreurs de validation des données.",
        "Les mises à jour ne sont pas appliquées correctement.",
        "La recherche dans l'application ne fonctionne pas.",
        "Le tableau de bord ne s'affiche pas.",
        "Les utilisateurs ne reçoivent pas leurs confirmations par email.",
        "Le chargement des images est très lent.",
        "Les permissions d'accès sont incorrectes.",
        "Le service client ne reçoit pas les messages."
    ],
    "label": [
        "critique", "moyen", "bas", "critique", "moyen", "bas", "critique", "bas",
        "critique", "moyen", "moyen", "critique", "bas", "moyen", "critique", "moyen",
        "critique", "bas", "critique", "critique", "moyen", "moyen", "moyen", "bas",
        "moyen", "moyen", "moyen", "moyen", "moyen", "moyen"
    ]
}

# Créer un DataFrame
df = pd.DataFrame(data)

# Sauvegarder en CSV
file_path = "notes_pannes.csv"
df.to_csv(file_path, index=False)
file_path

'notes_pannes.csv'

In [4]:
import pandas as pd 

df = pd.read_csv('notes_pannes.csv')
df, df.info(), df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   note    30 non-null     object
 1   label   30 non-null     object
dtypes: object(2)
memory usage: 608.0+ bytes


(                                                 note     label
 0           Le serveur est en panne, tout est bloqué.  critique
 1                  L'application est lente à charger.     moyen
 2       Une fonctionnalité mineure ne fonctionne pas.       bas
 3                   La base de données est corrompue.  critique
 4   La page de connexion prend du temps à s'afficher.     moyen
 5      Le bouton de téléchargement ne fonctionne pas.       bas
 6            Le réseau est complètement hors service.  critique
 7          Il y a un léger décalage dans l'affichage.       bas
 8          Le système redémarre de manière aléatoire.  critique
 9   Certains utilisateurs ne peuvent pas se connec...     moyen
 10       Les emails ne sont pas envoyés correctement.     moyen
 11  Le site web est inaccessible pour certains uti...  critique
 12        Les notifications push ne fonctionnent pas.       bas
 13      Les rapports ne sont pas générés comme prévu.     moyen
 14            Le paiemen

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Charger les données
data = pd.read_csv("notes_pannes.csv")

# Mapper les labels en entiers
label_mapping = {"critique": 0, "moyen": 1, "bas": 2}
data['label'] = data['label'].map(label_mapping)

# Diviser les données en ensembles d'entraînement et de test
train_texts, val_texts, train_labels, val_labels = train_test_split(data['note'].tolist(), data['label'].tolist(), test_size=0.2)

In [15]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset

# Charger le tokenizer et le modèle
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Préparer les datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

train_dataset = Dataset.from_dict({"input_ids": train_encodings['input_ids'], "attention_mask": train_encodings['attention_mask'], "labels": train_labels})
val_dataset = Dataset.from_dict({"input_ids": val_encodings['input_ids'], "attention_mask": val_encodings['attention_mask'], "labels": val_labels})

# Configurer les arguments de l'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=32,
    weight_decay=0.01,
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Créer un Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Entraîner le modèle
trainer.train()

# Sauvegarder le modèle fine-tuné
model.save_pretrained("./fine_tuned_model_classif")
tokenizer.save_pretrained("./fine_tuned_model_classif")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 2/64 [00:00<00:28,  2.17it/s]

{'eval_loss': 1.1187443733215332, 'eval_runtime': 0.042, 'eval_samples_per_second': 142.851, 'eval_steps_per_second': 23.809, 'epoch': 1.0}


  6%|▋         | 4/64 [00:01<00:28,  2.13it/s]

{'eval_loss': 1.0964492559432983, 'eval_runtime': 0.042, 'eval_samples_per_second': 142.852, 'eval_steps_per_second': 23.809, 'epoch': 2.0}


  9%|▉         | 6/64 [00:02<00:27,  2.07it/s]

...

 97%|█████████▋| 62/64 [00:31<00:00,  2.06it/s]

{'eval_loss': 0.9793005585670471, 'eval_runtime': 0.043, 'eval_samples_per_second': 139.534, 'eval_steps_per_second': 23.256, 'epoch': 31.0}


100%|██████████| 64/64 [00:32<00:00,  1.99it/s]


{'eval_loss': 0.9777347445487976, 'eval_runtime': 0.042, 'eval_samples_per_second': 142.855, 'eval_steps_per_second': 23.809, 'epoch': 32.0}
{'train_runtime': 32.093, 'train_samples_per_second': 23.93, 'train_steps_per_second': 1.994, 'train_loss': 0.6566612720489502, 'epoch': 32.0}


('./fine_tuned_model_classif\\tokenizer_config.json',
 './fine_tuned_model_classif\\special_tokens_map.json',
 './fine_tuned_model_classif\\vocab.txt',
 './fine_tuned_model_classif\\added_tokens.json')

In [16]:
# Évaluer le modèle
results = trainer.evaluate()
print(results)

100%|██████████| 1/1 [00:00<00:00, 200.00it/s]

{'eval_loss': 0.9777347445487976, 'eval_runtime': 0.04, 'eval_samples_per_second': 150.005, 'eval_steps_per_second': 25.001, 'epoch': 32.0}


In [19]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Charger le modèle fine-tuné
model = DistilBertForSequenceClassification.from_pretrained("./fine_tuned_model_classif")
tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_model_classif")

# Fonction pour classifier une nouvelle note
def classify_note(note):
    inputs = tokenizer(note, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities).item()
    return predicted_class

# Exemple d'utilisation
new_note = "il y a une baisse."
predicted_class = classify_note(new_note)
class_mapping = {0: "critique", 1: "moyen", 2: "bas"}
print(f"La note est classifiée comme : {class_mapping[predicted_class]}")

La note est classifiée comme : bas


In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
#Ce modèle est une version affinée du modèle DistilBERT pour classer les commentaires toxiques.
#https://huggingface.co/martin-ha/toxic-comment-model?text=I+like+you.+I+love+you

model_path = "martin-ha/toxic-comment-model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)
result = pipeline('assole.') 
print(result)

[{'label': 'toxic', 'score': 0.9288287162780762}]
